In [205]:
# GetOldTweets3 no longer works since twitter changed the endpoint. Only been an issue for 4 days, but may be fixed.
# save code incase the developers fix it.
# import GetOldTweets3 as got
# tweetCriteria = got.manager.TweetCriteria().setQuerySearch("realDonaldTrump").setMaxTweets(2)
# tweet = got.manager.TweetManager.getTweets(tweetCriteria)


In [311]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from pprint import pprint
import pandas as pd
import numpy as np


# Scraping Data from Twitter
The following block will use Selenium to scrape tweets from twitter

In [346]:



def infinity_scroll(driver,filename):
    try:
        for i in range(1000):
            
            soupObj = BeautifulSoup(driver.page_source,'html.parser')
            
            with open(filename,'a') as f: #remember to delete the file everytime you run it.
                f.write(str(soupObj))
            
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(2)

    except:
        return
        
    return


def scrape_tweets(twitter_handle,from_date,to_date,filename):
    
    url = f'https://twitter.com/search?q=(from%3A{twitter_handle})%20until%3A{to_date}%20since%3A{from_date}%20-filter%3Alinks%20-filter%3Areplies&src=typed_query&f=live'
    response = requests.get(url)
    driver = webdriver.Firefox()
    driver.get(url)
    time.sleep(10) 
    
    # scrolls down to the bottom of the page
    infinity_scroll(driver,filename)

    driver.quit()
    
    return

In [352]:
# twitter_handle = 'gsk'
# twitter_handle = 'sanofi'
# twitter handle = 'AstraZeneca'
# twitter_handle = 'realdonaldtrump'

twitter_handle = 'elonmusk'
from_date = '2019-09-24'
to_date = '2020-09-24'


scrape_tweets(twitter_handle,from_date,to_date,'elon_tweets_Sept_2019_Sept_2020.xml')

# Processing Tweet Data
The following code processes the data pulled from the XML file to create a listing of all Tweets.

In [353]:
def get_tweets(filename):

    with open(filename,'r') as f:
        soupObj = BeautifulSoup(f,'html.parser')
        
    num_tweets = len(soupObj.find_all('div',"css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0"))
    num_times = len(soupObj.find_all('time'))
    
    timestamp = soupObj.find('time')
    tweet = soupObj.find('div',"css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
    
    tweet_listing = [tweet.text]
    time_listing = [timestamp.attrs['datetime']]
    
    for i in range(max(num_tweets,num_times)):
        try:
            #pull timestamps
            time_listing.append((timestamp.find_next('time').attrs)['datetime'])
            timestamp = timestamp.find_next('time')
            
        except AttributeError:
            time_listing.append(np.NaN)
        try:
            # pull tweets
            tweet_listing.append(tweet.find_next('div',"css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0").text)
            tweet = tweet.find_next('div',"css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
        
        except AttributeError:
            tweet_listing.append(np.NaN)


    return pd.DataFrame({'Time':time_listing,'Tweet':tweet_listing})



In [355]:
# filepath = 'astrazeneca_tweets_Oct_2017_Sept_24_2020.xml'
# filepath = 'gsk_tweets_Oct_2017_Sept_24_2020.xml'
# filepath = 'pfizer_tweets_Oct_2017_Sept_24_2020.xml'
filepath = 'elon_tweets_Sept_2019_Sept_2020.xml'

df = get_tweets(filepath)

In [356]:
df.tail(20)

,Time,Tweet
286,2019-10-06T16:06:58.000Z,Customized horn & movement sounds (coconuts be...
287,2019-10-02T22:16:45.000Z,"Over 550,000 Tesla Smart Summon uses in first ..."
288,2019-09-27T19:25:10.000Z,Starship will allow us to inhabit other worlds
289,2019-09-26T14:57:34.000Z,Tesla V10.0 just started rolling out to all US...
290,2019-09-26T05:55:35.000Z,If advanced AI (beyond basic bots) hasn’t been...
291,2019-10-15T22:06:04.000Z,So much respect for those doing high volume ma...
292,2019-10-13T03:20:15.000Z,Is Space Jam good?
293,2019-10-12T01:01:26.000Z,"Now that Tesla V10.0 with Smart Summon is out,..."
294,2019-10-10T00:23:28.000Z,All Tesla Supercharger stations in regions aff...
295,2019-10-09T17:41:57.000Z,Tesla software V10.0 comes with karaoke


In [357]:
# df.to_csv('gsk_tweets_Oct_2017_Sept_24_2020.csv')
# df.to_csv('pfizer_tweets_Oct_2017_Sept_24_2020.csv')
# df.to_csv('astrazeneca_tweets_Oct_2017_Sept_24_2020.csv')
# df.to_csv('trump_tweets_Sep_2019_Sep_2020.csv')
df.to_csv('elon_tweets_Sept_2019_Sept_2020.csv')

In [ ]:
# variables for testing scrape_tweets()
# 'https://twitter.com/search?q=(from%3Arealdonaldtrump)%20until%3A2020-07-01%20since%3A2020-01-01%20-filter%3Alinks%20-filter%3Areplies&src=typed_query&f=live'
# twitter_handle = 'realdonaldtrump'
# from_date = '2020-01-01'
# to_date = '2020-07-01'

# other scrapers:
# snscrape's jsonl
# https://github.com/twintproject/twint/issues/918#issuecomment-696448934